In [3]:
from taming.modules.vqvae.quantize import VectorQuantizer2

In [4]:
import sys
print(sys.path)

['/root/app/taming-transformers', '/home/bruce/miniconda3/lib/python311.zip', '/home/bruce/miniconda3/lib/python3.11', '/home/bruce/miniconda3/lib/python3.11/lib-dynload', '', '/root/.local/lib/python3.11/site-packages', '/home/bruce/miniconda3/lib/python3.11/site-packages']


In [5]:
sys.path.append("./")

In [6]:
import torch

In [7]:
from taming.modules.diffusionmodules.model import Encoder

In [8]:
model = torch.load("/root/app/taming-transformers/logs/2020-11-03T15-34-24_imagenetdepth_vqgan/checkpoints/last.ckpt",map_location=torch.device('cuda:0'))

In [9]:
from omegaconf import OmegaConf
config_path = "/root/app/taming-transformers/logs/2020-11-03T15-34-24_imagenetdepth_vqgan/configs/2020-11-03T15-34-24-project.yaml"
config = OmegaConf.load(config_path)
print(config.model.params.ddconfig)

{'double_z': False, 'z_channels': 256, 'resolution': 256, 'in_channels': 1, 'out_ch': 1, 'ch': 128, 'ch_mult': [1, 1, 2, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [16], 'dropout': 0.0}


In [16]:
print('a.b.c'.split('.'))

['a', 'b', 'c']


In [21]:
enc_params = {'.'.join(key.split('.')[1:]): value for key,value in model['state_dict'].items() if key.startswith('encoder.')}
print(enc_params)

{'conv_in.weight': tensor([[[[-0.0448, -0.1371, -0.0495],
          [-0.0918,  0.3096, -0.2574],
          [-0.1342,  0.2851,  0.0617]]],


        [[[-0.2549,  0.1062, -0.2735],
          [ 0.1830,  0.0261, -0.0326],
          [-0.1869, -0.1946, -0.1186]]],


        [[[-0.0336, -0.1703,  0.0833],
          [-0.2234, -0.1310, -0.2066],
          [ 0.0289, -0.3230,  0.2668]]],


        ...,


        [[[-0.1547, -0.1650, -0.0671],
          [-0.2741,  0.0099,  0.2871],
          [ 0.2112,  0.1903,  0.3181]]],


        [[[ 0.0853, -0.3164,  0.1314],
          [-0.1252,  0.1284,  0.3472],
          [-0.1434,  0.1940, -0.1422]]],


        [[[-0.0504,  0.1956,  0.2781],
          [-0.0130,  0.3271,  0.0903],
          [-0.0730, -0.2599, -0.0813]]]], device='cuda:0'), 'conv_in.bias': tensor([-0.2192, -0.0675,  0.0533, -0.1114, -0.1869,  0.0236,  0.2852, -0.2379,
         0.0855, -0.1320, -0.1723,  0.1800, -0.2564, -0.0362, -0.0072, -0.2278,
         0.1040, -0.0039, -0.1384,  0.2173,  0.

In [22]:
enc = Encoder(**config.model.params.ddconfig)

enc.load_state_dict(enc_params)

<All keys matched successfully>

In [25]:
import cv2

In [30]:
im = cv2.imread("/root/app/taming-transformers/data/ade20k_segmentations/ADE_val_00000532.png")
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
im = torch.Tensor(im)
print(im.shape)

torch.Size([512, 778, 3])


In [31]:
res = enc(im)

RuntimeError: Given groups=1, weight of size [128, 1, 3, 3], expected input[1, 512, 778, 3] to have 1 channels, but got 512 channels instead